In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from torchvision import models




# There were issues with my certificate
import ssl
import certifi

ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())


/Users/lexele/Library/Python/3.11/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset Loading and Data Processing
Here we load the CIFAR10 dataset

In [ ]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

# batch_size = 4 # In the powerpoint slides it is 32?

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# #functions to show an image
# def imshow(img):
# 	img = img / 2 + 0.5 # unnormalize
# 	npimg = img.numpy()
# 	plt.imshow(np.transpose(npimg, (1,2,0)))
# 	plt.show()


# #get random training images
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# # show images
# imshow(torchvision.utils.make_grid(images))

# # print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))


In [2]:

def preprocess_data(transformations, batch_size):
	"""
	This function creates dataloaders with model-specific transformations for data preprocessing
	"""
	train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transformations)
	train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
	test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transformations)
	test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

	# Should we return train_set and test_loader too?
	return train_loader, test_loader

### Defining our CNNs
***
We will be experimenting with various pre-trained Deep CNNs - specifically ResNet and VGG architectures.

We will be using 2 different versions of Resnet and 2 different versions of VGG.

VGG: We will be using models `vgg11` , `vgg11_bn` (with batch normalization), `vgg19`, `vgg19_bn`

RESNET: We will be using `resnet18` (ResNet-18) and `resnet152` (ResNet-152)

** apply its specific preprocessing transformations to  CIFAR-10 dataset when creating DataLoader objects. **

For each model we use, we will:
1. Import, insantiate, and load it with default, pretrained weights
2. Get the correct preprocessing transformations for that specific model
3. Apply those transformations to our CIFAR10 dataset
4. Modify the classifer part of the model for our 10-class classification task.
-> this means that we need to freeze the ConvLayer(s) parts of the network, and modify the last couple layers -- follow Lecture 9 examples.


In [ ]:
def get_device():
	if torch.cuda.is_available():
		return torch. device ('cuda' )
	elif torch.backends.mps.is_available():
		return torch.device ('mps')
	else:
		return torch.device('cpu')
	
device = get_device()	
model = models.vgg16().to(device)


# summary(model.to(device), (3, 224, 224))

def better_summary(model, input_size, batch_size=32):
	"""
	A better summary function that ensures device consistency between weights used for the VGG model
	"""
	
	# Get the device the model is on
	model_on_cpu = model.to('cpu')
	summary(model_on_cpu, input_size, batch_size=batch_size)
	# Move model back to original device
	model.to(device)

better_summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 224, 224]           1,792
              ReLU-2         [32, 64, 224, 224]               0
            Conv2d-3         [32, 64, 224, 224]          36,928
              ReLU-4         [32, 64, 224, 224]               0
         MaxPool2d-5         [32, 64, 112, 112]               0
            Conv2d-6        [32, 128, 112, 112]          73,856
              ReLU-7        [32, 128, 112, 112]               0
            Conv2d-8        [32, 128, 112, 112]         147,584
              ReLU-9        [32, 128, 112, 112]               0
        MaxPool2d-10          [32, 128, 56, 56]               0
           Conv2d-11          [32, 256, 56, 56]         295,168
             ReLU-12          [32, 256, 56, 56]               0
           Conv2d-13          [32, 256, 56, 56]         590,080
             ReLU-14          [32, 256,

In [19]:
from torchvision.models import resnet18, resnet152, ResNet18_Weights, ResNet152_Weights
from torchvision.models import vgg11, vgg11_bn, vgg19, vgg19_bn, VGG11_Weights, VGG19_Weights

batch_size = 32
# Load ResNet models with pretrained weights
resnet18_model = resnet18(weights=ResNet18_Weights.DEFAULT)
resnet152_model = resnet152(weights=ResNet152_Weights.DEFAULT)

# Load VGG models with pretrained weights for transfer learning
vgg11_model = vgg11(weights=VGG11_Weights.DEFAULT)
vgg19_model = vgg19(weights=VGG19_Weights.DEFAULT)

# Get transformations for every model
resnet18_transforms = ResNet18_Weights.DEFAULT.transforms()
resnet152_transforms = ResNet152_Weights.DEFAULT.transforms()
vgg11_transforms = VGG11_Weights.DEFAULT.transforms()
vgg19_transforms = VGG19_Weights.DEFAULT.transforms()

models = [vgg11_model]


# Create dataloaders for each model with its specific transforms
resnet18_train_loader, resnet18_test_loader = preprocess_data(resnet18_transforms, batch_size=batch_size)
resnet152_train_loader, resnet152_test_loader = preprocess_data(resnet152_transforms, batch_size=batch_size)
vgg11_train_loader, vgg11_test_loader = preprocess_data(vgg11_transforms, batch_size=batch_size)
vgg19_train_loader, vgg19_test_loader = preprocess_data(vgg19_transforms, batch_size=batch_size) 
list_of_transformations = [resnet18_transforms, resnet152_transforms, vgg11_transforms, vgg19_transforms]

for model in models:
	better_summary(model.to(device), (3, 224, 224))
	# print(model)
	for param in model.features.parameters():
		param.requires_grad = False

	model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
	model.classifier = nn.Sequential(nn.Flatten(),
								  	nn.Linear(512, 128),
								 	nn.Linear(128, 1))
	
	better_summary(model.to(device), (3, 224, 224))

	


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 224, 224]           1,792
              ReLU-2         [32, 64, 224, 224]               0
         MaxPool2d-3         [32, 64, 112, 112]               0
            Conv2d-4        [32, 128, 112, 112]          73,856
              ReLU-5        [32, 128, 112, 112]               0
         MaxPool2d-6          [32, 128, 56, 56]               0
            Conv2d-7          [32, 256, 56, 56]         295,168
              ReLU-8          [32, 256, 56, 56]               0
            Conv2d-9          [32, 256, 56, 56]        

### Define a MLP
This is where the classifier is?

### Training, Experimentation and Transfer Learning

Doing transfer learning on ResNet is slightly different from VGG (the layers don't have the same names), so we print the networks to know which layers to freeze.